#Predicting the probability of a song beeing chosen by Obama in his playlists

##Data exploration

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from google.colab import auth

# Will collect your credentials
auth.authenticate_user()

In [ ]:
# Query Bigquery
query = "SELECT * FROM `music-recommendation-system-24.ml_tables_eu.song_list_obama_wo_duplicates_view`"
project = "music-recommendation-system-24"

df_full = pd.read_gbq(query=query, project_id=project)

<ipython-input-4-345f26455b79>:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_full = pd.read_gbq(query=query, project_id=project)


In [ ]:
df_full.head()

,acousticness,danceability,duration_min,energy,genre,genre2,instrumentalness,key,liveness,loudness,...,speechiness,track_album_name,track_album_release_year,track_artist,track_id,track_name,track_popularity,tempo,valence,in_obama_playlist
0,84.000,52.0,3.73,37.0,jazz,<NA>,0.0000,C,11.00,-9.350,...,3.00,Chet Baker Sings And Plays (Remastered 2004),1955,Chet Baker,0UZFhlhUMrYyzzyOHkylM3,Let's Get Lost,52,78,55.0,1
1,8.350,71.1,3.48,72.4,pop,<NA>,0.0000,G,3.89,-4.809,...,10.60,Blue,2018,Jonas Blue,14nIFzHQf1OBWf6VHVR8DF,Wherever You Go,21,104,47.8,0
2,0.236,38.2,3.91,69.8,pop,<NA>,0.0052,F#,10.80,-3.409,...,3.74,Love You Forever,2019,Nicky Romero,2gqzTQEn4g5ly0n5ScdMx0,Love You Forever,61,128,15.1,0
3,8.000,65.0,4.30,69.0,reggaeton,False,0.0000,D#,53.00,-5.750,...,4.00,Un Verano Sin Ti,2022,"Bad Bunny,Bomba Estéreo",3k3NWokhRRkEPhCzPmV8TW,Ojitos Lindos,81,80,27.0,1
4,22.700,68.9,3.05,68.6,rap,False,0.0000,E,9.82,-5.049,...,41.90,STEPZOLOGI,2019,Stepz,34FrYXSm9WiGCn7mT1lVuE,BeBe,52,120,66.9,0


In [ ]:
df_full.shape

(558, 21)

In [ ]:
df_full.isna().sum()

,0
acousticness,0
danceability,0
duration_min,0
energy,0
genre,0
genre2,327
instrumentalness,0
key,0
liveness,0
loudness,0


In [ ]:
df_full["in_obama_playlist"].value_counts()

,count
in_obama_playlist,
0,300
1,258


##Cleaning for ML

In [ ]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   acousticness              558 non-null    float64
 1   danceability              558 non-null    float64
 2   duration_min              558 non-null    float64
 3   energy                    558 non-null    float64
 4   genre                     558 non-null    object 
 5   genre2                    231 non-null    boolean
 6   instrumentalness          558 non-null    float64
 7   key                       558 non-null    object 
 8   liveness                  558 non-null    float64
 9   loudness                  558 non-null    float64
 10  mode                      558 non-null    object 
 11  speechiness               558 non-null    float64
 12  track_album_name          558 non-null    object 
 13  track_album_release_year  558 non-null    Int64  
 14  track_arti

In [ ]:
df = df_full.drop(columns=["track_name", "track_album_name"])

In [ ]:
df = df.set_index("track_id")

##Modeling

In [ ]:
from re import X
from sklearn.model_selection import train_test_split

# every columns in X variable except re_purchase which is our target
X = df.drop(columns=["in_obama_playlist"])
y = df["in_obama_playlist"]

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# store customers_ids for after
test_customers_ids = X_test.index

###Dealing with categorical features

In [ ]:
df.head(2)

,acousticness,danceability,duration_min,energy,genre,genre2,instrumentalness,key,liveness,loudness,mode,speechiness,track_album_release_year,track_artist,track_popularity,tempo,valence,in_obama_playlist
track_id,,,,,,,,,,,,,,,,,,
0UZFhlhUMrYyzzyOHkylM3,84.00,52.0,3.73,37.0,jazz,<NA>,0.0,C,11.00,-9.350,major,3.0,1955,Chet Baker,52,78,55.0,1
14nIFzHQf1OBWf6VHVR8DF,8.35,71.1,3.48,72.4,pop,<NA>,0.0,G,3.89,-4.809,major,10.6,2018,Jonas Blue,21,104,47.8,0


In [ ]:
X_train_cat = X_train[["track_artist", "genre", "key", "mode"]]
X_test_cat = X_test[["track_artist", "genre", "key", "mode"]]

In [ ]:
ohe = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown="ignore")

In [ ]:
X_train_cat_encoded = ohe.fit_transform(X_train_cat)
X_test_cat_encoded = ohe.transform(X_test_cat)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded, columns=ohe.get_feature_names_out())
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded, columns=ohe.get_feature_names_out())

In [ ]:
X_train_cat_encoded_df.head()

,"track_artist_2 Chainz,Ty Dolla $ign,Trey Songz,Jhené Aiko","track_artist_2Pac,Roger,Dr. Dre",track_artist_A Tribe Called Quest,track_artist_ATYPISK,track_artist_Accept,track_artist_Airbourne,track_artist_Akon,track_artist_Al Green,track_artist_Alessia Cara,track_artist_Alesso,...,key_C,key_C#,key_D,key_D#,key_E,key_F,key_F#,key_G,key_G#,mode_minor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_cat_encoded_df.shape

(446, 408)

###Dealing with numerical features

In [ ]:
df.head(2)

,acousticness,danceability,duration_min,energy,genre,genre2,instrumentalness,key,liveness,loudness,mode,speechiness,track_album_release_year,track_artist,track_popularity,tempo,valence,in_obama_playlist
track_id,,,,,,,,,,,,,,,,,,
0UZFhlhUMrYyzzyOHkylM3,84.00,52.0,3.73,37.0,jazz,<NA>,0.0,C,11.00,-9.350,major,3.0,1955,Chet Baker,52,78,55.0,1
14nIFzHQf1OBWf6VHVR8DF,8.35,71.1,3.48,72.4,pop,<NA>,0.0,G,3.89,-4.809,major,10.6,2018,Jonas Blue,21,104,47.8,0


In [ ]:
X_train_num = X_train[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]
X_test_num = X_test[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [ ]:
std = StandardScaler()

In [ ]:
X_train_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 446 entries, 5YLnfy7R2kueN0BRPkjiEG to 2w2nGYcTq7iP1Aj6AI8KIi
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_popularity          446 non-null    Int64  
 1   track_album_release_year  446 non-null    Int64  
 2   duration_min              446 non-null    float64
 3   danceability              446 non-null    float64
 4   energy                    446 non-null    float64
 5   loudness                  446 non-null    float64
 6   speechiness               446 non-null    float64
 7   acousticness              446 non-null    float64
 8   instrumentalness          446 non-null    float64
 9   liveness                  446 non-null    float64
 10  valence                   446 non-null    float64
 11  tempo                     446 non-null    Int64  
dtypes: Int64(3), float64(9)
memory usage: 62.8+ KB


In [ ]:
X_train_num_scaled = std.fit_transform(X_train_num)
X_test_num_scaled = std.transform(X_test_num)

In [ ]:
X_train_num_scaled_df = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns)
X_test_num_scaled_df = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns)

###Concatenating numerical and categorical features

In [ ]:
X_train_processed = pd.concat([X_train_cat_encoded_df, X_train_num_scaled_df], axis=1)
X_test_processed = pd.concat([X_test_cat_encoded_df, X_test_num_scaled_df], axis=1)

##Fitting our logistic regression model

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train_processed, y_train)

LogisticRegression()

In [ ]:
y_pred = logreg.predict(X_test_processed)

In [ ]:
logreg_accuracy = accuracy_score(y_test, y_pred)
logreg_accuracy

0.9017857142857143

##Predicting Obama's future musical taste with logistic regression

###Creating the song list

In [ ]:
# Query Bigquery
query = "SELECT * FROM `music-recommendation-system-24.ml_tables_eu.spotify_top_100_for_prediction_csv_V1`"
project = "music-recommendation-system-24"

df_full_prediction = pd.read_gbq(query=query, project_id=project)

<ipython-input-31-cfc479d20529>:5: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_full_prediction = pd.read_gbq(query=query, project_id=project)


In [ ]:
df_full_prediction.head(5)

,track_id,track_name,track_artist,track_popularity,track_album_name,track_album_release_year,duration_min,genre,record_label,danceability,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,3hRV0jL3vUpRrcy398teAU,The Night We Met,Lord Huron,88,Strange Trails,2015,3.47,folk,Play It Again Sam,44.8,...,A,-9.514,Major,4.52,96.80,27.20,63.8,9.98,174,3
1,0nrRP2bk19rLc0orkWPQk2,Wake Me Up,Avicii,83,TRUE,2013,4.12,edm,Universal Music AB,53.2,...,D,-5.697,Major,5.23,0.38,0.12,16.1,64.30,124,4
2,7BqHUALzNBTanL6OvsqmC1,Happier,"Marshmello,Bastille",82,Happier,2018,3.57,edm,Joytime Collective,68.7,...,F,-2.749,Major,4.52,19.10,0.00,16.7,67.10,100,4
3,5HCyWlXZPP0y6Gqq8TgA20,STAY (with Justin Bieber),"The Kid LAROI,Justin Bieber",78,STAY (with Justin Bieber),2021,2.36,pop,Columbia,59.1,...,C#,-5.484,Major,4.83,3.83,0.00,10.3,47.80,170,4
4,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,Taylor Swift,89,Lover,2019,2.97,pop,Taylor Swift,55.2,...,A,-5.707,Major,15.70,11.70,0.00,10.5,56.40,170,4


###Preparing the data for ML

In [ ]:
df_prediction = df_full_prediction.drop(columns=["track_name", "track_album_name"])

In [ ]:
df_prediction.loc[df_prediction["genre"] == "folk", "genre"] = "pop"

In [ ]:
df_prediction = df_prediction.set_index("track_id")

In [ ]:
df_prediction_cat = df_prediction[["track_artist", "genre", "key", "mode"]]
df_prediction_num = df_prediction[["track_popularity", "track_album_release_year", "duration_min", "danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]]

In [ ]:
df_prediction_cat_encoded = ohe.transform(df_prediction_cat)
df_prediction_num_scaled = std.transform(df_prediction_num)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
df_prediction_cat_encoded_df = pd.DataFrame(df_prediction_cat_encoded, columns=ohe.get_feature_names_out())
df_prediction_num_scaled_df = pd.DataFrame(df_prediction_num_scaled, columns=df_prediction_num.columns)

In [ ]:
df_prediction_processed = pd.concat([df_prediction_cat_encoded_df, df_prediction_num_scaled_df], axis=1)

###Predicting

In [ ]:
logreg_predictions = logreg.predict(df_prediction_processed)
logreg_predictions

array([1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 1.])

In [ ]:
proba = pd.DataFrame(logreg.predict_proba(df_prediction_processed), columns=["Obama dislikeability", "Obama likeability"])

In [ ]:
proba["Obama likeability"]=round(proba["Obama likeability"]*100,2)
proba["Obama dislikeability"]=round(proba["Obama dislikeability"]*100,2)

proba.head()

,Obama dislikeability,Obama likeability
0,18.12,81.88
1,92.25,7.75
2,94.26,5.74
3,72.29,27.71
4,68.31,31.69


In [ ]:
df_full_prediction_proba = df_full_prediction.merge(proba, left_index=True, right_index=True)

In [ ]:
df_full_prediction_proba = df_full_prediction_proba.sort_values(by="Obama likeability", ascending=False)
df_full_prediction_proba = df_full_prediction_proba[["track_name", "track_artist", "Obama likeability"]]
df_full_prediction_proba

,track_name,track_artist,Obama likeability
7,Memories,Maroon 5,86.27
30,I Wanna Be Yours,Arctic Monkeys,85.62
36,INDUSTRY BABY (feat. Jack Harlow),"Lil Nas X,Jack Harlow",84.65
14,Too Good At Goodbyes,Sam Smith,83.53
34,No Role Modelz,J. Cole,82.19
0,The Night We Met,Lord Huron,81.88
17,As It Was,Harry Styles,81.02
6,Without Me,Halsey,79.09
11,Someone Like You,Adele,78.76
33,Kill Bill,SZA,75.49
